<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1LeaguePrediccter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen

In [1]:
"hello world"

'hello world'

In [116]:
# 札幌 276
# 仙台 238 
def scrape(homeTeamID, awayTeamID, year):
    data = pd.DataFrame()
    
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0, 2, 4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)
    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(level=0, drop=True)
    df = df.rename(columns={'試合平均': 'homeTeamData','Unnamed: 3': 'Data','試合平均.1': 'awayTeamData'})
    df = df[['Data', 'homeTeamData', 'awayTeamData']]

    html = requests.get(url)
    html.encoding = "UTF-8"
    soup = BeautifulSoup(html.text, "html.parser")

    texts = soup.find('div', attrs={'class': 'gameTbl'}).text
    texts = re.findall(r'\w+', texts)
    
    return df

In [117]:
scrape('276', '238', '2021')

,Data,homeTeamData,awayTeamData
0,攻撃,17.83,11.29
1,パス,14.01,8.71
2,クロス,1.77,1.59
3,ドリブル,2.05,0.99
4,シュート,7.58,6.23
5,ゴール,3.39,2.42
6,奪取,106.54,91.51
7,守備,11.87,16.57
8,セーブ,0.41,0.64
9,攻撃回数,119.6,119.2


NameError: ignored